In [1]:
import nvidia.dali as dali
import nvidia.dali.types as types
import os
import argparse

if __name__ == "__main__":
    
    width = 640
    height = 480
    pipe = dali.pipeline.Pipeline(batch_size=256, num_threads=4, device_id=0)
    with pipe:
        images = dali.fn.external_source(device="gpu", name="DALI_INPUT_0")

        # skip to decode image from file
        # images = dali.fn.image_decoder(images, device="cpu", output_type=types.RGB)
#         shapes = dali.fn.cast(dali.fn.peek_image_shape(images), dtype=dali.types.FLOAT)  # shape is HWC
        shapes = dali.fn.shapes(images, dtype=types.INT64)
        origin_w = dali.fn.slice(shapes, 1, 1, axes=[0])  # extract width
        origin_h = dali.fn.slice(shapes, 0, 1, axes=[0])  # extract height
        scale_factors = min(width / origin_w, height / origin_h)
        
        images = dali.fn.resize(images, size=[height, width], mode='not_larger')
        images = dali.fn.crop(images, crop_w=width, crop_h=height, out_of_bounds_policy="pad")
        images = dali.fn.cast(images, dtype=types.FLOAT)
        images = dali.fn.transpose(images, perm=[2, 0, 1])
        pipe.set_outputs(images, scale_factors)
        pipe.serialize(filename="model.dali")

    print("Saved {}".format("/model.dali"))

TypeError: "DataNode" is a symbolic representation of TensorList used for defining graph of operations for DALI Pipeline. It should not be used for truth evaluation in regular Python context. Bool conversion in Pipeline can be achieved with "Cast" operator. To see what operations are allowed on DataNodes to represent computations in DALI Pipeline see the "Mathematical Expressions" section of DALI documentation.